In [75]:
from glob import glob
import os
import numpy as np
import torch
import torchvision
import torch.nn as nn
from torchvision import models
import torchvision.transforms as transforms
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda
from torch.utils.data import Dataset
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torchvision.utils import make_grid
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts
import torchvision.datasets as datasets


In [ ]:
path = '/Users/taehoonkim/Desktop/dogsandcats/'

files = glob(os.path.join(path,'*/*.jpg'))

print(f'Total no of images {len(files)}')

no_of_images = len(files)

shuffle = np.random.permutation(no_of_images)

os.mkdir(os.path.join(path, 'valid'))



In [ ]:
for t in ['train', 'valid']:
    for folder in ['dog/','cat/']:
        os.mkdir(os.path.join(path,t,folder))


In [9]:
for i in shuffle[:2000]:
    folder = files[i].split('/')[-1].split('.')[0]
    image = files[i].split('/')[-1]
    os.rename(files[i],os.path.join(path,'valid',folder,image))

## 경로 잘 확인해라 쫌!!

In [3]:
simple_transform = transforms.Compose([transforms.Resize((224,224))
                                       ,transforms.ToTensor()
                                       ,transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])
train = ImageFolder('/Users/taehoonkim/Desktop/dogsandcats',simple_transform)
valid = ImageFolder('/Users/taehoonkim/Desktop/dogsandcats',simple_transform)

In [ ]:
## train.class_to_idx - {'cat': 0, 'dog': 1}
## train.classes - ['cat', 'dog']

In [5]:
def imshow(inp):
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.255])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)

In [ ]:
imshow(train[50][0])

In [15]:
train_data_gen = torch.utils.data.DataLoader(train,batch_size=64,num_workers=3)
valid_data_gen = torch.utils.data.DataLoader(valid,batch_size=64,num_workers=3)

In [47]:
##print(torch.backends.mps.is_built())
##print(torch.backends.mps.is_available())
## torch.cuda.is_available()
## a = torch.tensor([1, 2, 3, 4]).to("mps")
## a
## torch.backends.mps.is_available()
## torch.cuda.is_available()
## torch.cuda.current_device()
is_cuda = torch.cuda.is_available()
device = torch.device('cuda' if is_cuda else 'cuda')

print('Current cuda device is', device)

Current cuda device is cuda


In [50]:

model_ft = models.resnet18(pretrained=True)
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs, 2)

if is_cuda:
    model_ft = model_ft.cuda()



In [101]:
learning_rate = 0.01
criterion = nn.CrossEntropyLoss()
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.01, momentum=0.9)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size = 7, gamma=0.01)



In [99]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=5):
    since = time.time()

    best_model_wts = model.state_dict()
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # 각 Epoch은 학습 단계와 검증 단계를 거침
        for phase in ['train', 'valid']:
            if phase == 'train':
                scheduler.step()
                model.train(True)  # 학습 모드 설정
            else:
                model.train(False)  # 검증 모드 설정

            running_loss = 0.0
            running_corrects = 0


            for data in dataloaders[phase]:
                inputs, labels = data
                if is_cuda:
                    inputs = Variable(inputs.cuda())
                    labels = Variable(labels.cuda())
                else:
                    inputs, labels = Variable(inputs), Variable(labels)

                optimizer.zero_grad()

                outputs = model(inputs)
                _, preds = torch.max(outputs.data, 1)
                loss = criterion(outputs, labels)

                if phase == 'train':
                    loss.backwart()
                    optimizer.step

                running_loss += loss.data[0]
                running_corrects += torch.sum(preds == labels.data)

            ipoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # 모델 복사(Deep Copy)
            if phase == 'valid' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = model.state_dict()

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    model.load_state_dict(best_model_wts)
    return model




In [103]:
train_model(model=learning_rate, criterion=criterion, optimizer=optimizer_ft, scheduler=exp_lr_scheduler)

NameError: name 'time' is not defined

In [14]:
print(files[0].split('/')[-1])

dog.8011.jpg


In [ ]:
## print(torch.backends.mps.is_available())
## print(torch.backends.mps.is_built())
## torch.cuda.is_available()
print(torch.__version__)
print(torch.backends.mps.is_built())
print(torch.backends.mps.is_available())
mps_device = torch.device("mps")
x = torch.ones(5, device=mps_device)
# 또는
x = torch.ones(5, device="mps")
y = x * 2